In [54]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [55]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [56]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    self.a = tf.Variable(initial_value=tf.zeros(shape=[2,14]), trainable=False, name= self.name + "_activation")
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1))   #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1))   #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1))   #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1))   #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1)))   #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1)))   #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1)))     #["Plu"]  [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1)))   #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1)))   #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1)))     #["Tr"]   [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1)))   #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1)))   #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1)))     #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1)))   #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1)))     #["Ri"]   [1,5] 

    output = tf.concat(to_hidden,axis = 0)
    output = tf.transpose(output)
    self.a.assign(output)
    return self.a.value()

In [57]:
class HiddenLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(HiddenLayer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[0]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
    net = tf.math.reduce_sum(inputs * self.w, axis=0) 
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()


In [58]:
class OutputLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(OutputLayer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[-1]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
     
    net = tf.matmul(inputs,self.w) 
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()

---

# CUSTOM MODEL

In [59]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self, learning_rate, initial_temperature, bias_value):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = HiddenLayer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=bias_value)
    self.o = OutputLayer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=bias_value)
    self.temperature = tf.Variable(initial_value = initial_temperature,trainable=False, name = "Temperature")
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.alpha = learning_rate

  def call(self, input):
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    return tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))

    ### BACK PROPAGATION
    
    # Compute temperature
    self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Ao = self.o.a * (self.h.a - self.o.a) * (1 - self.o.a) + (self.o.deltaA * ((self.h.a - self.o.a)) )
    delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao)
    self.o.deltaA.assign(delta_Ao)
    self.o.w.assign_add(delta_Wo) # DA AGGIORNARE DOPO INSIEME

    ## Update hidden weights
    delta_Ah = self.h.a * (1 - self.h.a) * tf.matmul(self.o.deltaA, self.o.w)
    delta_Wh = self.alpha * delta_Ah * self.input_layer.a
    self.h.deltaA.assign(delta_Ah)
    self.h.w.assign_add(delta_Wh)

    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value())
        }

In [60]:
# Construct an instance of CustomModel
input = tf.constant([0.01, 0.00, 0.03, 0.22, 0.87, 0.26, 0.42, 0.16])
model = MQSelfReflexiveNetwork(learning_rate=1.45, initial_temperature=9.0, bias_value=0.1)

In [61]:
model.compile()
model_history = model.fit(x = input, epochs=500, shuffle=False)

Epoch 1/500
1/1 [==============================] - 0s 272ms/step - Error: 0.1674 - Temperature: 0.1434 - Hidden weights: 0.4864 - Output weights: 0.5710
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - Error: 0.4905 - Temperature: 0.3291 - Hidden weights: 0.4871 - Output weights: 0.5762
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - Error: 1.4444 - Temperature: 0.5909 - Hidden weights: 0.4865 - Output weights: 0.5746
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - Error: 2.1547 - Temperature: 0.6830 - Hidden weights: 0.4868 - Output weights: 0.5751
Epoch 5/500
1/1 [==============================] - 0s 2ms/step - Error: 2.2983 - Temperature: 0.6968 - Hidden weights: 0.4865 - Output weights: 0.5747
Epoch 6/500
1/1 [==============================] - 0s 6ms/step - Error: 2.3174 - Temperature: 0.6986 - Hidden weights: 0.4865 - Output weights: 0.5745
Epoch 7/500
1/1 [==============================] - 0s 4ms/step - Error: 2.3198 - Temperature

In [62]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights
0,0.167350,0.143359,"[[0.49775252, 0.4835382, 0.49925226, 0.5, 0.48...","[[0.57202643, 0.57099724, 0.57048243, 0.569894..."
1,0.490524,0.329095,"[[0.4977971, 0.4844313, 0.49932143, 0.5, 0.481...","[[0.57307595, 0.57422066, 0.57662755, 0.581601..."
2,1.444428,0.590906,"[[0.49771824, 0.48367056, 0.4992822, 0.5, 0.48...","[[0.5729146, 0.57349324, 0.5749065, 0.57751465..."
3,2.154694,0.683012,"[[0.4977597, 0.4840037, 0.499298, 0.5, 0.48074...","[[0.57289135, 0.57364094, 0.57541394, 0.578995..."
4,2.298328,0.696816,"[[0.49771717, 0.48366922, 0.4992823, 0.5, 0.48...","[[0.5726494, 0.57329804, 0.5749003, 0.57801324..."
...,...,...,...,...
495,0.000000,0.000000,"[[0.46303728, 0.22588749, 0.48980325, 0.5, 0.1...","[[0.10362886, 0.22794834, 0.085417286, 0.05682..."
496,0.000000,0.000000,"[[0.46303728, 0.22588749, 0.48980325, 0.5, 0.1...","[[0.10362886, 0.22794834, 0.085417286, 0.05682..."
497,0.000000,0.000000,"[[0.46303728, 0.22588749, 0.48980325, 0.5, 0.1...","[[0.10362886, 0.22794834, 0.085417286, 0.05682..."
498,0.000000,0.000000,"[[0.46303728, 0.22588749, 0.48980325, 0.5, 0.1...","[[0.10362886, 0.22794834, 0.085417286, 0.05682..."


In [63]:
df_model_history.to_csv("model_history.csv")

In [64]:
dvf = abs(model.get_weights()[1][0] - model.get_weights()[1][1])
dvf

array([1.0349572 , 0.04983866, 0.2549113 , 0.02630493, 0.21558383,
       0.26208133, 0.06944798, 0.21646759, 0.20885512, 0.00866494,
       0.19305542, 0.01765224, 0.4463557 , 1.027682  ], dtype=float32)

In [65]:
dva = abs(model.get_weights()[0][0,:] - model.get_weights()[0][1,:])
dva

array([0.84000003, 0.03999999, 0.24999999, 0.03      , 0.16      ,
       0.26      , 0.06      , 0.22      , 0.21      , 0.01      ,
       0.19      , 0.02      , 0.39      , 0.86      ], dtype=float32)

In [66]:
dvf

array([1.0349572 , 0.04983866, 0.2549113 , 0.02630493, 0.21558383,
       0.26208133, 0.06944798, 0.21646759, 0.20885512, 0.00866494,
       0.19305542, 0.01765224, 0.4463557 , 1.027682  ], dtype=float32)

In [67]:
dvf = [round(x, 2) for x in dvf]

In [68]:
dvf_book = [0.97, 0.05, 0.29, 0.03, 0.17, 0.3, 0.06, 0.24, 0.24, 0.01, 0.21, 0.03, 0.43, 1.00]

In [69]:
df_diff = pd.DataFrame({"dva":dva, "dvf":dvf, "dvf_book":dvf_book})
df_diff

,dva,dvf,dvf_book
0,0.84,1.03,0.97
1,0.04,0.05,0.05
2,0.25,0.25,0.29
3,0.03,0.03,0.03
4,0.16,0.22,0.17
5,0.26,0.26,0.30
6,0.06,0.07,0.06
7,0.22,0.22,0.24
8,0.21,0.21,0.24
9,0.01,0.01,0.01


In [70]:
df_diff.to_csv("results.csv")

In [71]:
# standard deviation
df_diff.std()

dva         0.276390
dvf         0.336769
dvf_book    0.320581
dtype: float64

In [72]:
# % error std
((0.336769 - 0.320581) / 0.336769 ) * 100

4.806855737909363

In [73]:
fig = px.bar(df_diff,title="dva - dvf - dvf_book", barmode="group")
fig.update_layout( xaxis_title='dimension', yaxis_title='value')
fig.write_html("dva - dvf - dvf_book.html")
fig.show()

In [74]:
fig = px.bar(
    (df_diff["dvf"]-df_diff["dvf_book"]) / df_diff["dvf"],
    title="% differences between dvf and dvafbook",
    barmode="group",
    text_auto=".2",
    color = (df_diff["dvf"]-df_diff["dvf_book"])/df_diff["dvf"],
    color_continuous_scale="RdBu_r"
    )
fig.update_layout( xaxis_title='dimension', yaxis_title='% error')
fig.write_html("% differences between dvf and dvafbook.html")
fig.show()

In [75]:
import plotly.figure_factory as ff
hist_data = [((df_diff["dvf"]-df_diff["dvf_book"])/df_diff["dvf"]).to_list()]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,bin_size=0.2)
fig.update_layout(title_text="distribution of the % differences")
fig.write_html("Error and temperature.html")
fig.show()

In [76]:
fig = px.line(df_model_history[["Error","Temperature"]])
fig.update_layout(title="Error and temperature",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.write_html("Error and temperature.html")
fig.show()

---